# Getting Started with hpopt

This is quick start for hpoopt. You can learn how to use hpopt standalone with your own traninig module.

## 1. Installation


You only need to run following code to install hpopt.

``` bash
git clone https://github.com/openvinotoolkit/hyper_parameter_optimization
pip install -v -e ./hyper_parameter_optimization
```

## 2. Run HPO

First of all, you need to import hpopt to run HPO.

In [1]:
import hpopt

Then, make you own models. In this code, I implement very simple black-box function which has two hyper parameters 'a' and 'b'.

In [3]:

import random

def my_model(a, b, iter):
    return a**2 + (b - 10)**2 + iter * 0.1 + random.random() * 0.1

def my_trainer(config):
    for iter in range(config["iterations"]):
        score = my_model(**config["params"], iter=iter)
        if hpopt.report(config=config, score=score) == hpopt.Status.STOP:
            break

You should define hyper parameter search space to run HPO. Hyper parameter search space is dictionary which has hyper parameter name as key and search space sampling option as value.
Please refer [](./apis.md) for more details.

In [4]:
# make hyper parameter space
search_space = {"a": hpopt.search_space("uniform", [-5, 10]),
              "b": hpopt.search_space("uniform", [0, 15])}

Let's initiate hpopt class with hyper parameter search space we made. In this case, we run simple bayesian optimization(SMBO).

In [5]:
my_hpo = hpopt.create(save_path='./tmp/my_hpo', # where HPO progress is saved
                      search_alg="bayes_opt", # HPO algorithm
                      search_space=search_space,
                      early_stop="median_stop", # early stopping method
                      num_init_trials=5, num_trials=10, max_iterations=2,
                      resume=False,
                      num_full_iterations=2, # train iteration
                      full_dataset_size=1) # train dataset size. In this case, you can ignore it.

Finally, we can run HPO and print output.

In [6]:
while True:
    config = my_hpo.get_next_sample()

    if config is None:
        break

    my_trainer(config)

best_config = my_hpo.get_best_config()

my_hpo.print_results()

print("best hp: ", best_config)

|  #  |         a           |         b           |        score        |
|   1 |   9.629164679496109 |   9.152868582262837 |   93.56057080971651 |
|   2 |   9.503930895864768 |  10.477054966570938 |    90.6920277461489 |
|   3 |  6.5275823645277775 |   14.59670633381528 |   63.90848097562949 |
|   4 |  2.8808311670306797 |   9.058790892076829 |    9.29099328793063 |
|   5 |    7.95029121913541 |  13.178755341124605 |   73.50752311230228 |
|   6 |                10.0 |                 0.0 |  200.10740027553825 |
|   7 |   6.025026543316109 |                 0.0 |   136.4928912794813 |
|   8 |                10.0 |  1.6328830252328062 |  170.14644018670234 |
|   9 |                -5.0 |                 0.0 |  125.12112966688557 |
|  10 |   9.571587105736812 |0.004128397224773539 |  191.68849629266415 |
best hp:  {'a': 10.0, 'b': 0.0}


As you can see, hpopt found best hyper parameters for black box function.